In [ ]:
#imports 
import csv
import re
import pandas as pd

In [ ]:
#read in original davidson data
davidsonOG=pd.read_csv("davidson_labeled_data_copy.csv")
davidsonOG.head(3)

### Character Cleanup

Apply custom clean_tweets function to clean tweets of @tags and emoticon ASCII codes.  

In [ ]:
#present a good example for before and after cleaning
message = davidsonOG['tweet'][1556]
print(message)

In [ ]:
def clean_tweets(tweet): 
    tweet = re.sub('@[^ ]+ ', '', tweet) #remove user @ tags
    tweet = re.sub('&[^;]+;', '', tweet) #remove ascii codes for emojis
    tweet = tweet.replace('=', '', 1) #remove '=' at the beginning of some messages
    tweet = tweet.replace('\\n', ' ') #trim out \\n
    
    return tweet

In [ ]:
#apply clean_tweets custom function
davidsonOG['tweet'] = davidsonOG['tweet'].apply(clean_tweets)

#present cleaned tweet example from above
print(davidsonOG['tweet'][1556])

In [ ]:
davidsonOG = davidsonOG.loc[davidsonOG['tweet'] != ''] #remove any blank tweets
davidsonOG.shape

In [ ]:
davidsonOG.to_csv("davidson_labeled_data_replaced.csv", index=False) #create cleaned csv file
davidsonOG.head(3)

## Ambiguous Split

Class 0 = Hate Speech, Class 1 = Offensive Language, Class 2 = Neither

In [ ]:
davidsonReplaced = pd.read_csv('davidson_labeled_data_replaced.csv') #read cleaned tweets
keepCol = ['count', 'hate_speech', 'offensive_language', 'neither', 'class', 'tweet'] #created for abbreviation

In [ ]:
davidsonHate = davidsonReplaced.loc[(davidsonReplaced['offensive_language'] == 0) & (davidsonReplaced['neither'] == 0) \
                                    & (davidsonReplaced['class'] == 0)] #select tweets that were deemed universally hateful
davidsonHateTweets = davidsonHate[keepCol]
davidsonHateTweets.head(3)

In [ ]:
davidsonHateTweets.to_csv('DavidsonHateOnly.csv', index=True, header=True)

In [ ]:
davidsonHateMix = davidsonReplaced.loc[(davidsonReplaced['count'] != davidsonReplaced['hate_speech']) \
                                             & (davidsonReplaced['class'] == 0)] #select tweets that had a majority but not unanimous hate vote
davidsonHateMixTweets = davidsonHateMix[keepCol]
davidsonHateMixTweets.to_csv("DavidsonHateMix.csv", index=True, header=True)

In [ ]:
davidsonClean = davidsonReplaced.loc[(davidsonReplaced['offensive_language'] == 0) & (davidsonReplaced['hate_speech'] == 0) \
                                    & (davidsonReplaced['class'] == 2)] #select tweets that were deemed universally clean content
davidsonCleanTweets = davidsonClean[keepCol]
davidsonCleanTweets.to_csv('DavidsonCleanOnly.csv', index=True, header=True)